In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [6]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [9]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * (1-dones)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                    labels=tf.zeros_like(gQs2))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
#     ##################################################### repeatable!
#     next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
#                                    reuse=True)
#     next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
#                                 reuse=True)
#     gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
#                          action_size=action_size, reuse=True)
#     dones2 = tf.concat(axis=0, values=[dones[1:], tf.ones(shape=[1])])
#     gQs3 = tf.reshape(gQs3, shape=[-1]) * (1-dones2)
#     dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
#                                                                     labels=tf.zeros_like(gQs3))) # 0-1
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
#                                                                      labels=tf.ones_like(gQs3))) # 0-1
    return actions_logits, aloss, dloss, aloss2

In [10]:
def model_opt(a_loss, a_loss2, d_loss, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [11]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, #model input 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        a_loss2=self.a_loss2, 
                                                        d_loss=self.d_loss,
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [12]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [13]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [14]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [15]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
gamma = 0.99                   # future reward discount

In [16]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, d_learning_rate=d_learning_rate, gamma=gamma)

# Init the memory
memory = Memory(max_size=memory_size)

In [17]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            rate = -1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0:
                    # print('np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0: ', 
                    #       np.count_nonzero(dones), len(dones), np.max(rates))
                    break
            if np.count_nonzero(dones)!=1 and len(dones) < 1 and np.max(rates) <= 0:
                print(np.count_nonzero(dones), len(dones), np.max(rates))
                break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                feed_dict = {model.states: states, 
                                            model.actions: actions,
                                            model.next_states: next_states,
                                            model.rewards: rewards,
                                            model.dones: dones,
                                            model.rates: rates})
            dloss, _ = sess.run([model.d_loss, model.d_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print('dones:', 
            #       len(dones), np.count_nonzero(dones), 
            #       len(dones1), np.count_nonzero(dones1), 
            #       len(dones2), np.count_nonzero(dones2))
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:20.0000 R:20.0000 rate:0.0400 aloss:1.3854 dloss:2.1510 aloss2:1.3346 exploreP:0.9980
Episode:1 meanR:21.0000 R:22.0000 rate:0.0440 aloss:1.3836 dloss:2.0366 aloss2:1.4015 exploreP:0.9959
Episode:2 meanR:18.6667 R:14.0000 rate:0.0280 aloss:1.3969 dloss:2.0017 aloss2:1.4238 exploreP:0.9945
Episode:3 meanR:23.7500 R:39.0000 rate:0.0780 aloss:1.3867 dloss:1.9881 aloss2:1.4246 exploreP:0.9906
Episode:4 meanR:26.4000 R:37.0000 rate:0.0740 aloss:1.3884 dloss:1.9940 aloss2:1.4154 exploreP:0.9870
Episode:5 meanR:24.3333 R:14.0000 rate:0.0280 aloss:1.4002 dloss:1.8716 aloss2:1.4867 exploreP:0.9857
Episode:6 meanR:22.2857 R:10.0000 rate:0.0200 aloss:1.3857 dloss:2.0821 aloss2:1.3779 exploreP:0.9847
Episode:7 meanR:21.5000 R:16.0000 rate:0.0320 aloss:1.3903 dloss:1.9713 aloss2:1.4260 exploreP:0.9831
Episode:8 meanR:20.4444 R:12.0000 rate:0.0240 aloss:1.3942 dloss:1.8837 aloss2:1.4778 exploreP:0.9820
Episode:9 meanR:22.0000 R:36.0000 rate:0.0720 aloss:1.3832 dloss:1.9119 aloss2:1.4

Episode:80 meanR:26.9012 R:15.0000 rate:0.0300 aloss:1.3780 dloss:0.6556 aloss2:4.5774 exploreP:0.8062
Episode:81 meanR:26.7073 R:11.0000 rate:0.0220 aloss:1.3725 dloss:0.7146 aloss2:4.3366 exploreP:0.8053
Episode:82 meanR:26.6627 R:23.0000 rate:0.0460 aloss:1.3905 dloss:0.6545 aloss2:4.4493 exploreP:0.8035
Episode:83 meanR:26.5000 R:13.0000 rate:0.0260 aloss:1.3798 dloss:0.6918 aloss2:4.5049 exploreP:0.8024
Episode:84 meanR:26.6471 R:39.0000 rate:0.0780 aloss:1.3869 dloss:0.6706 aloss2:4.6189 exploreP:0.7993
Episode:85 meanR:26.5465 R:18.0000 rate:0.0360 aloss:1.3813 dloss:0.6562 aloss2:4.5725 exploreP:0.7979
Episode:86 meanR:26.3908 R:13.0000 rate:0.0260 aloss:1.3787 dloss:0.6205 aloss2:4.7897 exploreP:0.7969
Episode:87 meanR:26.2045 R:10.0000 rate:0.0200 aloss:1.3974 dloss:0.6563 aloss2:4.9299 exploreP:0.7961
Episode:88 meanR:26.0337 R:11.0000 rate:0.0220 aloss:1.3761 dloss:0.6109 aloss2:4.6617 exploreP:0.7953
Episode:89 meanR:26.8556 R:100.0000 rate:0.2000 aloss:1.3862 dloss:0.6379

Episode:159 meanR:38.7000 R:87.0000 rate:0.1740 aloss:1.4011 dloss:0.5374 aloss2:6.2789 exploreP:0.5814
Episode:160 meanR:39.7700 R:121.0000 rate:0.2420 aloss:1.4382 dloss:0.5428 aloss2:6.2744 exploreP:0.5745
Episode:161 meanR:40.4800 R:100.0000 rate:0.2000 aloss:1.4145 dloss:0.5583 aloss2:6.1958 exploreP:0.5689
Episode:162 meanR:41.2600 R:102.0000 rate:0.2040 aloss:1.4331 dloss:0.5424 aloss2:6.2496 exploreP:0.5632
Episode:163 meanR:41.7600 R:67.0000 rate:0.1340 aloss:1.4282 dloss:0.5642 aloss2:6.2165 exploreP:0.5595
Episode:164 meanR:42.7700 R:119.0000 rate:0.2380 aloss:1.4267 dloss:0.5543 aloss2:6.1689 exploreP:0.5530
Episode:165 meanR:43.0400 R:45.0000 rate:0.0900 aloss:1.4450 dloss:0.5265 aloss2:6.2069 exploreP:0.5506
Episode:166 meanR:44.2900 R:146.0000 rate:0.2920 aloss:1.4656 dloss:0.5296 aloss2:6.2744 exploreP:0.5427
Episode:167 meanR:44.5800 R:102.0000 rate:0.2040 aloss:1.4467 dloss:0.5356 aloss2:6.3018 exploreP:0.5373
Episode:168 meanR:45.1600 R:94.0000 rate:0.1880 aloss:1.43

Episode:238 meanR:105.7900 R:149.0000 rate:0.2980 aloss:1.4174 dloss:0.6148 aloss2:5.8893 exploreP:0.2320
Episode:239 meanR:105.9800 R:96.0000 rate:0.1920 aloss:1.4217 dloss:0.6262 aloss2:5.8985 exploreP:0.2299
Episode:240 meanR:110.3900 R:452.0000 rate:0.9040 aloss:1.4119 dloss:0.6100 aloss2:5.9159 exploreP:0.2202
Episode:241 meanR:111.6000 R:135.0000 rate:0.2700 aloss:1.4099 dloss:0.6143 aloss2:5.9413 exploreP:0.2174
Episode:242 meanR:113.3700 R:262.0000 rate:0.5240 aloss:1.4239 dloss:0.6269 aloss2:5.9186 exploreP:0.2120
Episode:243 meanR:115.0200 R:188.0000 rate:0.3760 aloss:1.4119 dloss:0.6700 aloss2:5.8017 exploreP:0.2082
Episode:244 meanR:116.0200 R:114.0000 rate:0.2280 aloss:1.4236 dloss:0.5826 aloss2:5.8295 exploreP:0.2060
Episode:245 meanR:116.5600 R:87.0000 rate:0.1740 aloss:1.4032 dloss:0.6025 aloss2:5.8844 exploreP:0.2043
Episode:246 meanR:116.6800 R:94.0000 rate:0.1880 aloss:1.3991 dloss:0.6236 aloss2:5.8981 exploreP:0.2025
Episode:247 meanR:119.3600 R:313.0000 rate:0.6260

Episode:316 meanR:187.5300 R:177.0000 rate:0.3540 aloss:1.3936 dloss:0.8762 aloss2:5.0112 exploreP:0.0554
Episode:317 meanR:187.6300 R:184.0000 rate:0.3680 aloss:1.3880 dloss:0.8138 aloss2:5.0165 exploreP:0.0546
Episode:318 meanR:191.3000 R:500.0000 rate:1.0000 aloss:1.3968 dloss:0.8306 aloss2:5.0259 exploreP:0.0524
Episode:319 meanR:194.5200 R:436.0000 rate:0.8720 aloss:1.3908 dloss:0.8778 aloss2:4.9986 exploreP:0.0506
Episode:320 meanR:199.3500 R:500.0000 rate:1.0000 aloss:1.3762 dloss:0.9436 aloss2:4.8924 exploreP:0.0486
Episode:321 meanR:201.6800 R:343.0000 rate:0.6860 aloss:1.3837 dloss:0.9205 aloss2:4.8185 exploreP:0.0473
Episode:322 meanR:201.9100 R:126.0000 rate:0.2520 aloss:1.3863 dloss:0.9108 aloss2:4.7835 exploreP:0.0469
Episode:323 meanR:203.8500 R:321.0000 rate:0.6420 aloss:1.3912 dloss:0.9051 aloss2:4.8064 exploreP:0.0457
Episode:324 meanR:207.0900 R:500.0000 rate:1.0000 aloss:1.3879 dloss:0.8731 aloss2:4.8270 exploreP:0.0440
Episode:325 meanR:210.8300 R:500.0000 rate:1.0

...

Episode:862 meanR:205.5000 R:201.0000 rate:0.4020 aloss:1.0889 dloss:0.9798 aloss2:5.5393 exploreP:0.0100
Episode:863 meanR:205.0800 R:73.0000 rate:0.1460 aloss:1.0899 dloss:0.9874 aloss2:5.2162 exploreP:0.0100
Episode:864 meanR:200.1800 R:10.0000 rate:0.0200 aloss:1.0868 dloss:1.0194 aloss2:4.9251 exploreP:0.0100
Episode:865 meanR:200.3300 R:366.0000 rate:0.7320 aloss:1.0900 dloss:0.9772 aloss2:5.3791 exploreP:0.0100
Episode:866 meanR:199.6900 R:149.0000 rate:0.2980 aloss:1.0890 dloss:0.9763 aloss2:5.5053 exploreP:0.0100
Episode:867 meanR:200.0000 R:41.0000 rate:0.0820 aloss:1.0872 dloss:0.9797 aloss2:5.4694 exploreP:0.0100
Episode:868 meanR:201.4100 R:244.0000 rate:0.4880 aloss:1.0905 dloss:0.9758 aloss2:5.6461 exploreP:0.0100
Episode:869 meanR:196.5100 R:10.0000 rate:0.0200 aloss:1.0955 dloss:0.9698 aloss2:6.2216 exploreP:0.0100
Episode:870 meanR:191.6200 R:11.0000 rate:0.0220 aloss:1.0987 dloss:0.9428 aloss2:5.7106 exploreP:0.0100
Episode:871 meanR:192.0500 R:237.0000 rate:0.4740 a

Episode:940 meanR:197.5700 R:53.0000 rate:0.1060 aloss:1.0783 dloss:0.9546 aloss2:5.7977 exploreP:0.0100
Episode:941 meanR:193.5500 R:10.0000 rate:0.0200 aloss:1.0786 dloss:0.9500 aloss2:5.8018 exploreP:0.0100
Episode:942 meanR:189.4900 R:27.0000 rate:0.0540 aloss:1.0794 dloss:0.9721 aloss2:6.0285 exploreP:0.0100
Episode:943 meanR:189.4900 R:10.0000 rate:0.0200 aloss:1.0812 dloss:0.9931 aloss2:5.6906 exploreP:0.0100
Episode:944 meanR:189.4900 R:500.0000 rate:1.0000 aloss:1.0797 dloss:0.9538 aloss2:6.2089 exploreP:0.0100
Episode:945 meanR:189.6000 R:21.0000 rate:0.0420 aloss:1.0808 dloss:1.0368 aloss2:5.2713 exploreP:0.0100
Episode:946 meanR:184.7000 R:10.0000 rate:0.0200 aloss:1.0752 dloss:0.9810 aloss2:5.9952 exploreP:0.0100
Episode:947 meanR:184.6100 R:10.0000 rate:0.0200 aloss:1.0781 dloss:1.0066 aloss2:6.2730 exploreP:0.0100
Episode:948 meanR:183.6200 R:10.0000 rate:0.0200 aloss:1.0840 dloss:0.9491 aloss2:6.4466 exploreP:0.0100
Episode:949 meanR:182.3600 R:374.0000 rate:0.7480 alos

Episode:1018 meanR:176.3400 R:500.0000 rate:1.0000 aloss:1.0815 dloss:1.0005 aloss2:6.1583 exploreP:0.0100
Episode:1019 meanR:175.6700 R:10.0000 rate:0.0200 aloss:1.0820 dloss:0.9691 aloss2:6.3282 exploreP:0.0100
Episode:1020 meanR:173.3000 R:10.0000 rate:0.0200 aloss:1.0834 dloss:0.9965 aloss2:5.9653 exploreP:0.0100
Episode:1021 meanR:173.2900 R:10.0000 rate:0.0200 aloss:1.0782 dloss:1.0135 aloss2:5.7542 exploreP:0.0100
Episode:1022 meanR:178.1900 R:500.0000 rate:1.0000 aloss:1.0789 dloss:0.9884 aloss2:6.2124 exploreP:0.0100
Episode:1023 meanR:183.0900 R:500.0000 rate:1.0000 aloss:1.0809 dloss:0.9865 aloss2:6.2095 exploreP:0.0100
Episode:1024 meanR:183.0900 R:500.0000 rate:1.0000 aloss:1.0782 dloss:0.9804 aloss2:6.3302 exploreP:0.0100
Episode:1025 meanR:183.0900 R:10.0000 rate:0.0200 aloss:1.0794 dloss:0.9743 aloss2:6.4051 exploreP:0.0100
Episode:1026 meanR:183.5100 R:71.0000 rate:0.1420 aloss:1.0829 dloss:0.9965 aloss2:6.3913 exploreP:0.0100
Episode:1027 meanR:178.9600 R:45.0000 rate

Episode:1096 meanR:153.9200 R:10.0000 rate:0.0200 aloss:1.0756 dloss:0.9214 aloss2:6.8375 exploreP:0.0100
Episode:1097 meanR:156.6500 R:283.0000 rate:0.5660 aloss:1.0760 dloss:0.9138 aloss2:6.7038 exploreP:0.0100
Episode:1098 meanR:156.6900 R:14.0000 rate:0.0280 aloss:1.0715 dloss:0.9121 aloss2:6.8535 exploreP:0.0100
Episode:1099 meanR:160.6700 R:429.0000 rate:0.8580 aloss:1.0741 dloss:0.9179 aloss2:6.7735 exploreP:0.0100
Episode:1100 meanR:155.7700 R:10.0000 rate:0.0200 aloss:1.0797 dloss:0.8966 aloss2:7.3638 exploreP:0.0100
Episode:1101 meanR:160.6700 R:500.0000 rate:1.0000 aloss:1.0760 dloss:0.9180 aloss2:6.8310 exploreP:0.0100
Episode:1102 meanR:160.0000 R:10.0000 rate:0.0200 aloss:1.0779 dloss:0.9402 aloss2:6.3937 exploreP:0.0100
Episode:1103 meanR:160.2600 R:36.0000 rate:0.0720 aloss:1.0794 dloss:0.8842 aloss2:7.1818 exploreP:0.0100
Episode:1104 meanR:160.2700 R:11.0000 rate:0.0220 aloss:1.0726 dloss:0.9057 aloss2:6.4946 exploreP:0.0100
Episode:1105 meanR:156.6700 R:65.0000 rate:

Episode:1174 meanR:178.8500 R:10.0000 rate:0.0200 aloss:1.0692 dloss:0.8921 aloss2:7.0492 exploreP:0.0100
Episode:1175 meanR:173.9500 R:10.0000 rate:0.0200 aloss:1.0671 dloss:0.9279 aloss2:7.3459 exploreP:0.0100
Episode:1176 meanR:174.1800 R:33.0000 rate:0.0660 aloss:1.0693 dloss:0.8887 aloss2:7.2515 exploreP:0.0100
Episode:1177 meanR:174.1800 R:10.0000 rate:0.0200 aloss:1.0741 dloss:0.8742 aloss2:7.1751 exploreP:0.0100
Episode:1178 meanR:174.1800 R:10.0000 rate:0.0200 aloss:1.0774 dloss:0.8979 aloss2:7.0893 exploreP:0.0100
Episode:1179 meanR:175.2800 R:269.0000 rate:0.5380 aloss:1.0774 dloss:0.8837 aloss2:7.3002 exploreP:0.0100
Episode:1180 meanR:175.2800 R:10.0000 rate:0.0200 aloss:1.0708 dloss:0.8777 aloss2:6.8944 exploreP:0.0100
Episode:1181 meanR:170.3800 R:10.0000 rate:0.0200 aloss:1.0707 dloss:0.8706 aloss2:7.3823 exploreP:0.0100
Episode:1182 meanR:170.3800 R:10.0000 rate:0.0200 aloss:1.0824 dloss:0.8890 aloss2:7.0368 exploreP:0.0100
Episode:1183 meanR:169.1500 R:10.0000 rate:0.

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.